In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# STEP 2: Install libraries
!pip install -U albumentations
!pip install opencv-python

In [ ]:
# STEP 3 : Verifikasi instalasi albumentations
import albumentations as A
import cv2
print("Albumentations versi",A.__version__)
print("OpenCV versi",cv2.__version__)

In [ ]:
# STEP 4: Import libraries
import os
import cv2
import random
import shutil
import albumentations as A
from google.colab import drive

In [ ]:
# STEP 5: Setup paths
image_dir = '/content/drive/MyDrive/train/images'
label_dir = '/content/drive/MyDrive/train/labels'

output_base = '/content/drive/MyDrive/augmented'
output_train_img = os.path.join(output_base, 'train/images')
output_train_lbl = os.path.join(output_base, 'train/labels')
output_val_img = os.path.join(output_base, 'val/images')
output_val_lbl = os.path.join(output_base, 'val/labels')

# Buat folder output
for path in [output_train_img, output_train_lbl, output_val_img, output_val_lbl]:
    os.makedirs(path, exist_ok=True)

In [ ]:
# STEP 6: Start Augmentation

# Fungsi baca label YOLO
def read_yolo_label(label_path):
    if not os.path.exists(label_path):
        return [], []
    with open(label_path, 'r') as f:
        lines = f.readlines()
    bboxes = []
    class_labels = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            cls, x, y, w, h = map(float, parts)
            bboxes.append([x, y, w, h])
            class_labels.append(int(cls))
    return bboxes, class_labels

# Simpan label ke file
def save_yolo_label(path, bboxes, class_labels):
    with open(path, 'w') as f:
        for box, cls in zip(bboxes, class_labels):
            x, y, w, h = box
            f.write(f"{int(cls)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# Fungsi augmentasi
def apply_aug(image, bboxes, class_labels, transform):
    transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
    return transformed['image'], transformed['bboxes'], transformed['class_labels']

# Augmentasi yang dilakukan
def get_augmentations():
    return {
        'rot90': A.Rotate(limit=(90, 90), p=1),     # Rotasi 90 derajat
        'rot180': A.Rotate(limit=(180, 180), p=1),  # Rotasi 180 derajat
        'rot270': A.Rotate(limit=(270, 270), p=1),  # Rotasi 270 derajat
        'hflip': A.HorizontalFlip(p=1),             # Flip Horizontal
        'vflip': A.VerticalFlip(p=1),               # Flip Vertikal
        'bright': A.RandomBrightnessContrast(p=1),  # Contrast Adjustment
    }

# Ambil semua file gambar
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

all_augmented = []

# Proses setiap gambar
for img_file in image_files:
    img_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')

    image = cv2.imread(img_path)
    bboxes, class_labels = read_yolo_label(label_path)

    name_base = os.path.splitext(img_file)[0]

    # Data asli
    all_augmented.append((image, bboxes, class_labels, f'{name_base}_orig'))

    # Augmentasi
    augmentations = get_augmentations()
    for aug_name, aug_op in augmentations.items():
        composed = A.Compose([aug_op], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
        img_aug, boxes_aug, cls_aug = apply_aug(image, bboxes, class_labels, composed)
        all_augmented.append((img_aug, boxes_aug, cls_aug, f'{name_base}_{aug_name}'))

# Acak dan split 80:20
random.shuffle(all_augmented)
split_idx = int(0.8 * len(all_augmented))
train_data = all_augmented[:split_idx]
val_data = all_augmented[split_idx:]

# Simpan data ke folder
def save_dataset(data_list, img_dir, lbl_dir):
    for img, boxes, clss, name in data_list:
        img_path = os.path.join(img_dir, f'{name}.jpg')
        lbl_path = os.path.join(lbl_dir, f'{name}.txt')
        cv2.imwrite(img_path, img)
        save_yolo_label(lbl_path, boxes, clss)

save_dataset(train_data, output_train_img, output_train_lbl)
save_dataset(val_data, output_val_img, output_val_lbl)

print(f"✅ Selesai! Total data: {len(all_augmented)}")
print(f"📁 Train: {len(train_data)}")
print(f"📁 Val: {len(val_data)}")